In [ ]:
import geopandas
import shapely
from shapely import geometry, ops
import geoapis
import pathlib
from geoapis import vector
import dotenv
import os

# Paths for files to open and to save

In [ ]:
file_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wairewa')
catchment_path = file_path / r"catchments\upper_catchment\upper_catchment.zip"
bathy_path = file_path / r"Lake\bathy_from_niwa\bathy_LakeForsyth.zip"
lake_name = file_path / r"Lake\processed\lake_outline.geojson"
bathy_name = file_path / r"Lake\processed\bathymetry.geojson"
land_name = file_path / r"catchments\upper_catchment_land.geojson"

Bathymetry from NIWA surveys. Provided by Sanjay. See email.

In [ ]:
bathy_in = geopandas.read_file(bathy_path)

Create a lake outline from the good zero contour

In [ ]:
lake_outline=bathy_in[bathy_in['OBJECTID_1']==801].reset_index(drop=True)
lake_outline=lake_outline.set_geometry([shapely.geometry.Polygon(lake_outline.loc[0].geometry)])
lake_outline.to_file(lake_name)

Drop the zero contours and keep the rest as Bathymetry data

In [ ]:
bathy_out = bathy_in[bathy_in['contour'] > 0].reset_index(drop=True)
bathy_out.to_file(bathy_name)

Read in the catchment and subtract the lake outline to produce the 'land' polygon

In [ ]:
catchment_in = geopandas.read_file(catchment_path)

In [ ]:
land=geopandas.overlay(catchment_in, lake_outline, how="difference")
land.to_file(land_name)

In [ ]:
(bathy_out.geometry.type=='LineString').any()

In [ ]:
import matplotlib
f, ax = matplotlib.pyplot.subplots(figsize=(30, 30))
ax=matplotlib.pyplot.subplot(111)
bathy_out.plot(ax=ax)

# Try download vectors of lakes from LINZ
Had to go to private NIWA data in the end

In [ ]:
lake_layer = 4781
dotenv.load_dotenv()
linz_key = os.environ.get('LINZ_API', None)

In [ ]:
vector_fetcher = geoapis.vector.Linz(linz_key,bounding_polygon=None,verbose=True)
vector = vector_fetcher.run(lake_layer)

# Track down cause of error in GeoFabrics

In [ ]:
land_name = file_path / r"catchments\clipped_contours.geojson"

In [ ]:
cb=geopandas.read_file(land_name)

In [ ]:
cb.plot()

In [ ]:
cb

In [ ]:
cb.geometry.type